# 데이터 전처리

@author 컴퓨터과학과 황승현

-----

- 결측값 47개 이상인 거는 제외해야한다!
- 코드를 쓰긴 했는데 너무 중구난방이다! 리펙토링이 필요하다!
- 내일 하겠다!

-----

## 2024년 New 전처리 행동강령

1. 데이터 drop은 최대한 줄인다.
1. 바이올린 플롯 등으로 이상값을 찾아 없앤다.
1. 결측값 대치는 변수 하나하나 고민해서 한다.

## 전처리 절차

1. 종속변수 AS1_OP 가공
    - AS1_DT
    - AS1_MT

2. 데이터셋 분리
    - 여성(폐경 전) (AS1_PMYN_C: 1)
    - 여성(폐경 후) (AS1_PMYN_C: 2)

3. var_to_use활용
    1. 질환유무, 약물력 변수 가공
    2. 가족력, 가족과의 관계 변수 가공
        - 부, 모, 형제자매, 기타 -> 부모, 형제자매, 기타, 해당없음

6. 이상치 탐지 후 제거: 바이올린 플롯

7. 결측값 대치
    - 전처리 행동강령

-----

독립변수는 4가지로 분류 (binary, cath0, cath1, cnt)

- binary: 범주형(binary) 변수
-  catH0: 계층 없는 범주형(>03) 변수, 벡터화(cb-encoding)
-  catH1: 계층 있는 범주형 변수
-    cnt: 연속형 변수, 정규화

## 준비

- 라이브러리 불러오기
- 데이터셋, 사용할 변수 불러오기

In [2]:
# 라이브러리 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import QuantileTransformer

In [3]:
# 데이터셋 불러오기
# dataset = pd.read_csv("dataset/Dataset_OP_230316_raw.csv", index_col=0, na_values=[77777, 99999, '#NULL!', ' ']) # 사용안함
# dataset = pd.read_csv("dataset/Dataset_OP_230330_raw.csv", index_col=0, na_values=[77777, 99999, '#NULL!', ' ']) # 사용안함
# dataset = pd.read_csv("dataset/Dataset_OP_230417_raw.csv", index_col=0, na_values=[77777, 99999, '#NULL!', ' ']) # 사용안함
# dataset = pd.read_csv("dataset/Dataset_OP_230424_raw.csv", index_col=0, na_values=[77777, 99999, '#NULL!', ' ']) # 사용안함
dataset = pd.read_csv("dataset/Dataset_OP_230614_raw.csv", index_col=0, na_values=[77777, 99999, '#NULL!', ' '], low_memory=False)
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9249 entries, NIH2307795046 to NIH2307534318
Columns: 838 entries, AS1_SEX to AS1_EGFR
dtypes: float64(795), int64(38), object(5)
memory usage: 59.2+ MB


In [4]:
# 특성별로 분류
# var_to_use = pd.read_csv("dataset/var_to_use_2023-03-30T2240I.csv")
# var_to_use = pd.read_csv("dataset/var_to_use_2023-04-21T1258I.csv")
# var_to_use = pd.read_csv("dataset/var_to_use_2023-04-24T1830I.csv")
# var_to_use = pd.read_csv("dataset/var_to_use_2023-04-28T1610I.csv")
var_to_use = pd.read_csv("dataset/var_to_use_2023-06-13T1650I.csv")
var_to_use


,binary,cath0,cath1,cnt,dependent
0,AS1_SEX,AS1_EDUA,AS1_INCOME,AS1_AGE,AS1_DT
1,AS1_HT,AS1_DRINK,AS1_PHYSTB,AS1_TOTALC,AS1_MT
2,AS1_DM,AS1_SMOKEA,AS1_PHYSIT,AS1_PMAG_C,NaN
3,AS1_AL,AS1_FMOSREL_N,AS1_PHYACTL,AS1_MNSAG,NaN
4,AS1_MI,AS1_FMOSREL_O,AS1_PHYACTM,AS1_CHILD_P,NaN
...,...,...,...,...,...
70,NaN,NaN,NaN,AS1_BPSIT1RS,NaN
71,NaN,NaN,NaN,AS1_BPSIT1SYS,NaN
72,NaN,NaN,NaN,AS1_BPSIT1LD,NaN
73,NaN,NaN,NaN,AS1_BPSIT1RD,NaN


In [5]:
print(f"전체 여성 수: {len(dataset[dataset['AS1_SEX'] == 2])}")

전체 여성 수: 4798


## 데이터 가공

- 기존 데이터로 새로운 데이터 만들기
- if 문 쓰면 안됨!
    - The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
- np.where 사용할 것!

### 종속변수 결측값 drop X

In [6]:
# def set_OP(row):
#     if pd.isna(row['AS1_DT']) and pd.isna(row['AS1_MT']):
#         return np.nan
#     elif not pd.isna(row['AS1_DT']) and row['AS1_DT'] <= -2.5:
#         return 1
#     elif not pd.isna(row['AS1_MT']) and row['AS1_MT'] <= -2.5:
#         return 1
#     else:
#         return 0

import pandas as pd
import numpy as np

def set_OP(row):
  """
  This function assigns a value of 1 to the 'OP' column if either 'AS1_DT' or 'AS1_MT' 
  is less than or equal to -2.5, and NaN otherwise.

  Args:
      row (pandas.Series): A row from the DataFrame.

  Returns:
      int: 1 if condition is met, NaN otherwise.
  """
  try:
    # Attempt to convert AS1_DT and AS1_MT to numeric (if possible)
    as1_dt_numeric = pd.to_numeric(row['AS1_DT'], errors='coerce')
    as1_mt_numeric = pd.to_numeric(row['AS1_MT'], errors='coerce')
  except:
    # If conversion fails (non-numeric values), return NaN for 'OP'
    return np.nan

  if pd.isna(as1_dt_numeric) and pd.isna(as1_mt_numeric):
    return np.nan
  elif not pd.isna(as1_dt_numeric) and as1_dt_numeric <= -2.5:
    return 1
  elif not pd.isna(as1_mt_numeric) and as1_mt_numeric <= -2.5:
    return 1
  else :
    return 0


In [7]:
dataset[dataset[['AS1_DT', 'AS1_MT']].isna().all(axis=1)]

,AS1_SEX,AS1_AGE,AS1_FAMNUM,AS1_HAND,AS1_SEAS,AS1_MARRYA,AS1_MARRYAETC,AS1_RELIG,AS1_RELIGETC,AS1_RELIGPRT,...,AS1_INSM.1,AS1_SLPHRD.1,AS1_SLPAMSF.1,AS1_ICOFF_1,AS1_ICOFF_2,AS1_W098.1,AS1_MNSAG.2,AS1_BRSTF,AS1_BRSTC,AS1_EGFR
DIST_ID,,,,,,,,,,,,,,,,,,,,,


In [8]:
# 종속변수의 결측값이 있으면 drop
test = dataset.dropna(subset=['AS1_DT', 'AS1_MT'])

# 남성(1) drop, 여성(2)만 가져옴
test = test[test['AS1_SEX'] == 2]
dataset = dataset[dataset['AS1_SEX'] == 2]

# 새로운 종속변수 만들기 OP: 골다공증 여부
test['OP'] = np.where((test['AS1_DT'] <= -2.5) | (test['AS1_MT'] <= -2.5), 1, 0)
dataset['OP'] = dataset.apply(set_OP, axis=1)

In [9]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4798 entries, NIH2307713334 to NIH2307534318
Columns: 839 entries, AS1_SEX to OP
dtypes: float64(795), int64(39), object(5)
memory usage: 30.7+ MB


In [10]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4680 entries, NIH2307713334 to NIH2307534318
Columns: 839 entries, AS1_SEX to OP
dtypes: float64(795), int32(1), int64(38), object(5)
memory usage: 30.0+ MB


In [11]:
print(len(test['OP']))
print(test['OP'].value_counts())

print(len(dataset['OP']))
print(dataset['OP'].value_counts())


4680
OP
0    3785
1     895
Name: count, dtype: int64
4798
OP
0    3874
1     924
Name: count, dtype: int64


종속변수의 결측값이 있으면 drop하는 것 비교

`dataset.dropna(subset=['AS1_DT', 'AS1_MT'], inplace=True)`

drop 함
```
4680
0    3785
1     895
Name: OP, dtype: int64
```

drop 안 함
```
4798
0    3874
1     924
Name: OP, dtype: int64
```

drop을 하지 않고 118명 더 확보


### 각종 질환 가공

- 질환 유무
    - 각종 종양 유무 (AS1_TOTCA1)
- 약물력
- 가족력

In [12]:
# 질환유무 가공
disease_list = np.array(['HT', 'DM', 'AL', 'MI', 'TH', # 질환 목록
                         'CH', 'CD', 'LP', 'AS', 'CL',
                         'KD', 'CV', 'GT' ]) # UL, PV, HN, TOTCA1 제외

dataset['AS1_ARRM'] = np.where((dataset['AS1_JOAR'] == 2) | (dataset['AS1_JORM'] == 2) | (dataset['AS1_TRTAR'] == 2), 1, 0) # 관절염은 조건 3가지 만족하면 1 아니면 0

for x in disease_list:
    dataset['AS1_'+x] = np.where((dataset['AS1_PD'+x] == 2) | (dataset['AS1_TRT'+x] == 2), 1, 0) # 질환여부 정의. AS1_PDㅇㅇ 또는 AS1_TRTㅇㅇ 이 2이면 1 아니면 0


In [13]:
# 약물력 가공
drug_list = np.array(['ST', 'CP', 'INS', 'HT', 'AR', 
                      'TH', 'FH', 'OS', 'STK', 'AS',
                      'LP', 'SP', 'SL'])  # UL, DM, CP, HP 제외 SP, SL 추가
for x in drug_list:
    dataset['AS1_DR'+x] = np.where((dataset['AS1_DRUG'+x] == 2) | (dataset['AS1_DRUG'+x+'CU'] == 2), 1, 0)


In [14]:
# 가족력 가공
# N: 가족력 없음, P: parent 부모, S: sibling 형제자매, O: other 기타
# N: REL1A~4A까지 99999
# P: REL1A~4A중 1, 2 있을 때
# S: REL1A~4A중 3 있을 때
# O: REL1A~4A중 4 있을 때

# disease_dict = {
#     'HT': 4, 'DM': 4, 'CVA': 3, 'HE': 2, 'OS': 2,
#     'CVB': 2, 'CD': 2, 'PV': 2, 'LP': 2, 'GT': 2,'CH': 1
# }
disease_dict = { 'OS': 2 }

for disease, num_family in disease_dict.items():
    dataset[f'AS1_FM{disease}REL_N'] = np.all(dataset[[f'AS1_FM{disease}REL{i}A' for i in range(1, num_family+1)]] == 99999, axis=1).astype(int)
    dataset[f'AS1_FM{disease}REL_P'] = np.any(dataset[[f'AS1_FM{disease}REL{i}A' for i in range(1, num_family+1)]] <= 2, axis=1).astype(int)
    dataset[f'AS1_FM{disease}REL_S'] = np.any(dataset[[f'AS1_FM{disease}REL{i}A' for i in range(1, num_family+1)]] == 3, axis=1).astype(int)
    dataset[f'AS1_FM{disease}REL_O'] = np.any(dataset[[f'AS1_FM{disease}REL{i}A' for i in range(1, num_family+1)]] == 4, axis=1).astype(int)

In [15]:
dataset[[f'AS1_FM{disease}REL_{family}' for disease in disease_dict.keys() for family in ['N', 'P', 'S', 'O']]]

,AS1_FMOSREL_N,AS1_FMOSREL_P,AS1_FMOSREL_S,AS1_FMOSREL_O
DIST_ID,,,,
NIH2307713334,0,0,0,0
NIH2307455306,0,0,0,0
NIH2307137554,0,0,0,0
NIH2307649667,0,0,0,0
NIH2307899915,0,1,0,0
...,...,...,...,...
NIH2307591031,0,0,0,0
NIH2307808901,0,0,0,0
NIH2307930134,0,0,0,0


### 여성력 가공

- ~~질환~~
- 약물
- 기타 수술 (AS1_OBGYOP)
    - 자궁경부암수술/ 자궁암수술/ 자궁내막암/자궁경부암,나팔관제거수술/ 만 1 나머지는 0

In [16]:
# 여성력 가공
# 남성 데이터 셋을 제거하여 필요없음

# 남자일 경우: 0 여자일 경우: 그대로
# for x in ['CP', 'FH']:
#     dataset['AS1_DR'+x] = np.where(dataset['AS1_SEX'] == '1',
#                                    0,
#                                    np.where((dataset['AS1_DRUG'+x] == 2) | (dataset['AS1_DRUG'+x+'CU'] == 2), 2, 1))

for x in ['CP', 'FH']:
    dataset['AS1_DR'+x] = np.where((dataset['AS1_DRUG'+x] == 2) | (dataset['AS1_DRUG'+x+'CU'] == 2), 1, 0)


# women = np.array(['AS1_PREG', 'AS1_HYST', 'AS1_HYSTOVARYW', 'AS1_OVARYW',
#                   'AS1_OBGYOP', 'AS1_BRCA', 'AS1_DRCP', 'AS1_DRFH'])
#
# for x in women:
#     dataset[x] = np.where((dataset['AS1_SEX'] == 1), 0, dataset[x])

### 앉은 자세 혈압
- AS1_BPSit1SYS
    - AS1_BPSit1LS, AS1_BPSit1RS 변수 중 높은 값
- AS1_BPSit1DIA
    - AS1_BPSit1LD,  AS1_BPSit1RD  변수 중 높은 값

In [17]:
dataset['AS1_BPSIT1SYS'] = np.where(dataset['AS1_BPSIT1LS'] > dataset['AS1_BPSIT1RS'], dataset['AS1_BPSIT1LS'], dataset['AS1_BPSIT1RS'])

dataset['AS1_BPSIT1DIA'] = np.where(dataset['AS1_BPSIT1LD'] > dataset['AS1_BPSIT1RD'], dataset['AS1_BPSIT1LD'], dataset['AS1_BPSIT1RD'])

### 타입 별 분리

In [18]:
# var_to_use의 각 열 가져옴

raw_binary = dataset.reindex(columns=(var_to_use['binary'].dropna()))
raw_cath0 = dataset.reindex(columns=(var_to_use['cath0'].dropna()))
raw_cath1 = dataset.reindex(columns=(var_to_use['cath1'].dropna()))
raw_cnt = dataset.reindex(columns=(var_to_use['cnt'].dropna()))
raw_label = dataset.reindex(columns=(var_to_use['dependent'].dropna()))

In [19]:
# null인 행이 있다.... 코드북과 데이터셋이 불일치한다.
dataframes = {
    "raw_binary": raw_binary,
    "raw_cath0": raw_cath0,
    "raw_cath1": raw_cath1,
    "raw_cnt": raw_cnt
}

for name, df in dataframes.items():
    null_cell = pd.Series(index=df.columns)
    null_cell = null_cell[df.isnull().sum() >= len(dataset)]  # 코드북에 없는 데이터 리스트
    print(f"{name}에서 발견된 누락 데이터 열:")
    print(null_cell)

raw_binary에서 발견된 누락 데이터 열:
Series([], dtype: float64)
raw_cath0에서 발견된 누락 데이터 열:
Series([], dtype: float64)
raw_cath1에서 발견된 누락 데이터 열:
Series([], dtype: float64)
raw_cnt에서 발견된 누락 데이터 열:
Series([], dtype: float64)


In [20]:
# dataset['OP']는 문제 없음
print(raw_label.isnull().sum())
print(dataset['OP'].isnull().sum())

dependent
AS1_DT    77
AS1_MT    41
dtype: int64
0


## 이상치 탐지, 결측값 대치

- 아주 썩어빠진 마인드
> - 일부 데이터의 많은 결측값
>   - 77777 -> 0
>     - AS1_TOTALC
>     - AS1_HVSMAM
>   - 99999 -> ?
>     - AS1_FMOSREL1A
>     - AS1_CA_U
>     - AS1_NA_U
>     - AS1_K_U
>
> - binary, cath0, cath1: 각 칼럼의 최빈값으로 대치
> - cnt: 평균값으로 대치

- 올바른 결측값 처리: 변수 하나하나 신경써서 하는 것.

-----

- 위에거는 다 무시
- 결측값 많은 것은 다 없애자!
- 기존 4798명 기준으로 1% 이상 (결측치 48 이상) 다 제거하고 돌려보기 

In [21]:
# 육체활동시간 살려달라
print(dataset['AS1_PHYACTH'].value_counts())
print(dataset['AS1_PHYACTH'].isnull().sum())

AS1_PHYACTH
0.0    2922
8.0     838
1.0     212
2.0     184
5.0     143
3.0     118
7.0     117
6.0     107
4.0      75
Name: count, dtype: int64
82


In [22]:
phys = ['AS1_PHYACTH', 'AS1_PHYACTL', 'AS1_PHYACTM', 'AS1_PHYSIT', 'AS1_PHYSTB']

In [23]:
dataset[phys] = dataset[phys].fillna(0.0)

In [24]:
dataset['AS1_PHYACTH'].isnull().sum()

0

In [25]:
dataset.head()

,AS1_SEX,AS1_AGE,AS1_FAMNUM,AS1_HAND,AS1_SEAS,AS1_MARRYA,AS1_MARRYAETC,AS1_RELIG,AS1_RELIGETC,AS1_RELIGPRT,...,AS1_DRAS,AS1_DRLP,AS1_DRSP,AS1_DRSL,AS1_FMOSREL_N,AS1_FMOSREL_P,AS1_FMOSREL_S,AS1_FMOSREL_O,AS1_BPSIT1SYS,AS1_BPSIT1DIA
DIST_ID,,,,,,,,,,,,,,,,,,,,,
NIH2307713334,2,44,2.0,1.0,2.0,2.0,NaN,4.0,NaN,NaN,...,0,0,0,0,0,0,0,0,100.0,70.0
NIH2307455306,2,43,6.0,1.0,2.0,2.0,NaN,3.0,NaN,1.0,...,0,0,0,0,0,0,0,0,120.0,80.0
NIH2307137554,2,64,4.0,1.0,1.0,2.0,NaN,3.0,NaN,1.0,...,0,0,0,0,0,0,0,0,174.0,102.0
NIH2307649667,2,65,2.0,1.0,3.0,2.0,NaN,2.0,NaN,2.0,...,0,0,0,0,0,0,0,0,126.0,82.0
NIH2307899915,2,57,6.0,1.0,3.0,2.0,NaN,3.0,NaN,2.0,...,0,0,0,0,0,1,0,0,102.0,74.0


In [26]:
 # AS1_BRCA 유방암 수술 여부.. 수술 안 함으로 변경
print(len(raw_binary['AS1_BRCA']))
print(raw_binary['AS1_BRCA'].isnull().sum())
raw_binary['AS1_BRCA'].value_counts()

4798
257


AS1_BRCA
1.0    4506
2.0      35
Name: count, dtype: int64

In [27]:
raw_binary['AS1_BRCA'] = raw_binary['AS1_BRCA'].fillna(1.0)
print(raw_binary['AS1_BRCA'].isnull().sum())
raw_binary['AS1_BRCA'].value_counts()

0


AS1_BRCA
1.0    4763
2.0      35
Name: count, dtype: int64

In [28]:
# AS1_PREG : 임신여부
def preg2fillna(df):
    mask = (df.AS1_BRSTF == 2) & df.AS1_PREG.isnull()

    df.loc[mask, 'AS1_PREG'] = 2

    return df


In [29]:
raw_binary = preg2fillna(raw_binary.copy())

In [30]:
# AS1_PMAG_C
print(len(raw_cnt['AS1_PMAG_C']))
print(raw_cnt['AS1_PMAG_C'].isnull().sum())
raw_cnt['AS1_PMAG_C'].value_counts()

4798
2099


AS1_PMAG_C
49.0    285
50.0    276
48.0    239
47.0    201
51.0    178
52.0    170
46.0    154
45.0    144
44.0    112
40.0     99
54.0     96
53.0     95
41.0     85
42.0     84
43.0     75
39.0     68
38.0     64
55.0     53
56.0     29
37.0     29
35.0     27
57.0     23
36.0     21
32.0     15
30.0     14
33.0     14
34.0     12
58.0     11
31.0     10
28.0      6
29.0      4
59.0      3
25.0      2
60.0      1
Name: count, dtype: int64

In [31]:
def pmagc2fillna(df, bin):
  # AS1_PMAG_C 칼럼의 결측값을 NaN으로 변경합니다.
  df.AS1_PMAG_C = df.AS1_PMAG_C.fillna(np.nan)

  # AS1_PMYN_C가 1인 경우 결측값을 0으로 대체합니다.
  df.loc[bin.AS1_PMYN_C == 1, 'AS1_PMAG_C'] = 0

  # AS1_PMYN_C가 2인 경우의 평균값을 계산합니다.
  mean_as1_pmag_c_2 = df.loc[bin.AS1_PMYN_C == 2, 'AS1_PMAG_C'].mean().astype(int)

  # AS1_PMYN_C가 2이고 AS1_PMAG_C 값이 결측값인 행을 선택합니다.
  mask = (bin.AS1_PMYN_C == 2) & df.AS1_PMAG_C.isnull()

  # 선택된 행의 AS1_PMAG_C 값을 평균값으로 대체합니다.
  df.loc[mask, 'AS1_PMAG_C'] = mean_as1_pmag_c_2

  return df

raw_cnt = pmagc2fillna(raw_cnt.copy(), raw_binary)


In [32]:
# AS1_PMAG_C 대치 완료, 폐경여부 알 수 없는 28명은 DROP
print(f"AS1_PMAG_C: {raw_cnt['AS1_PMAG_C'].isnull().sum()}, AS1_PMYN_C: {raw_binary['AS1_PMYN_C'].isnull().sum()}")
raw_cnt['AS1_PMAG_C'].value_counts()

AS1_PMAG_C: 28, AS1_PMYN_C: 28


AS1_PMAG_C
0.0     1738
46.0     487
49.0     285
50.0     276
48.0     239
47.0     201
51.0     178
52.0     170
45.0     144
44.0     112
40.0      99
54.0      96
53.0      95
41.0      85
42.0      84
43.0      75
39.0      68
38.0      64
55.0      53
37.0      29
56.0      29
35.0      27
57.0      23
36.0      21
32.0      15
30.0      14
33.0      14
34.0      12
58.0      11
31.0      10
28.0       6
29.0       4
59.0       3
25.0       2
60.0       1
Name: count, dtype: int64

In [33]:
raw_binary = raw_binary.dropna(subset=['AS1_PMYN_C'])
raw_cnt = raw_cnt.dropna(subset=['AS1_PMAG_C'])

### 일정 개수 이상 결측치 제거

- 결측값 많은 것은 다 없애자!
- 기존 4798명 기준으로 1% 이상 (결측치 48 이상) 다 제거하고 돌려보기 

In [34]:
def drop_columns_with_missing_values(df, threshold=47):
  """Drops columns with more than the specified threshold of missing values,
  excluding columns in the 'exception' list.

  Args:
      df (pandas.DataFrame): The input DataFrame.
      threshold (int, optional): The minimum number of missing values a column
          can have to be dropped. Defaults to 47.

  Returns:
      tuple: A tuple containing the modified DataFrame and the list of dropped columns.
  """

  missing_values_count = df.isnull().sum()
  columns_to_drop = missing_values_count[missing_values_count > threshold].index.tolist()

  # Exclude columns in the 'exception' list from being dropped
  exception = ['AS1_SMOKEA', 'AS1_INCOME', 'AS1_PHYSTB', 'AS1_PHYSIT', 'AS1_PHYACTL', 'AS1_PHYACTM', 'AS1_PHYACTH']
  columns_to_drop = [col for col in columns_to_drop if col not in exception]

  # Informative message about dropped columns (if any)
  if columns_to_drop:
    print(f"Dropped columns with more than {threshold} missing values: {', '.join(columns_to_drop)}")
  else:
    print("No columns dropped based on missing value threshold.")

  return df.drop(columns=columns_to_drop), columns_to_drop


# def drop_columns_with_missing_values(df, threshold=47):
#   missing_values_count = df.isnull().sum()
#   columns_to_drop = missing_values_count[missing_values_count > threshold].index.tolist()

#   # 민경누나 부탁으로 꼭 넣어야하는 거 다시 넣기
#   exception = ['AS1_SMOKEA', 'AS1_INCOME', 'AS1_PHYSTB', 'AS1_PHYSIT', 'AS1_PHYACTL', 'AS1_PHYACTM', 'AS1_PHYACTH'],
#   # `list.remove` 대신 `list.pop` 사용
#   # for col in exception:
#   #   if col in columns_to_drop:
#   #     columns_to_drop.pop(columns_to_drop.index(col))

#   columns_to_drop = [col for col in columns_to_drop if col not in exception]

#   print(df.columns[0], columns_to_drop)
#   df = df.drop(columns=columns_to_drop)

#   return df, columns_to_drop

  

### 남은거는 그냥 처리

In [35]:
# raw_binary = raw_binary.fillna(raw_binary.mode().iloc[0])
# raw_cath0 = raw_cath0.fillna(raw_cath0.mode().iloc[0])
# raw_cath1 = raw_cath1.fillna(raw_cath1.mode().iloc[0])
# raw_cnt = raw_cnt.fillna(raw_cnt.mean())

binary, dropped_col_binary = drop_columns_with_missing_values(raw_binary, 47)
binary = binary.fillna(binary.mode().iloc[0])

cath0, dropped_col_cath0 = drop_columns_with_missing_values(raw_cath0, 47)
cath0 = cath0.fillna(cath0.mode().iloc[0])

cath1, dropped_col_cath1 = drop_columns_with_missing_values(raw_cath1, 47)
cath1 = cath1.fillna(cath1.mode().iloc[0])

cnt, dropped_col_cnt = drop_columns_with_missing_values(raw_cnt, 47)
cnt = cnt.fillna(cnt.mean())

Dropped columns with more than 47 missing values: AS1_HYST, AS1_OBGYOP, AS1_BRSTF
Dropped columns with more than 47 missing values: AS1_HYSTOVARYW, AS1_OVARYW
No columns dropped based on missing value threshold.
Dropped columns with more than 47 missing values: AS1_CHILD_P, AS1_FPREGAG, AS1_VITB12, AS1_PRT_U, AS1_CREATINE_U, AS1_CA_U, AS1_NA_U, AS1_TSH


In [36]:
print(f"""
      binary: {len(binary)}
      cath0: {len(cath0)}
      cath1: {len(cath1)}        
      cnt: {len(cnt)}

      """)


      binary: 4770
      cath0: 4798
      cath1: 4798        
      cnt: 4770

      


In [37]:
print(dropped_col_binary)
print(dropped_col_cath0)
print(dropped_col_cath1)
print(dropped_col_cnt)

['AS1_HYST', 'AS1_OBGYOP', 'AS1_BRSTF']
['AS1_HYSTOVARYW', 'AS1_OVARYW']
[]
['AS1_CHILD_P', 'AS1_FPREGAG', 'AS1_VITB12', 'AS1_PRT_U', 'AS1_CREATINE_U', 'AS1_CA_U', 'AS1_NA_U', 'AS1_TSH']


In [38]:
# cath0['']

### 바이올린 플롯

In [39]:
# import seaborn as sns
# import matplotlib.pyplot as plt

In [40]:
# plt.tight_layout()
# # 75개의 바이올린 플롯을 위한 빈 Figure와 Axes 객체 생성
# fig, axes = plt.subplots(nrows=5, ncols=15, figsize=(50, 30))

# # 칼럼 인덱스 카운터
# i = 0

# # 모든 칼럼 순회
# for col in cnt.columns:
#     # 5행 15열 Axes 객체 배열에 순서대로 바이올린 플롯 그리기
#     sns.violinplot(data=cnt, x=col, ax=axes[i // 15][i % 15])
#     i += 1

# # 그래프 제목 설정
# fig.suptitle("[Continuous variables]")

# # x축 라벨 설정
# for ax in axes.flat:
#     ax.set_xlabel(ax.get_xlabel(), rotation=0)

# # y축 라벨 설정
# axes[0, 0].set_ylabel("[none]")

# # plt.show()를 사용하면 그래프가 즉시 뜨고 코드 실행이 멈춥니다.
# # plt.savefig("[파일 경로]")를 사용하면 지정된 경로에 그래프 파일을 저장합니다.
# plt.savefig("./Docs/drop_vp.png")

## 데이터셋에 유전체 붙이기

유전체 인덱스 매칭 안 되는 것 drop
유전체 값 0 0(결측값)인 것 drop

In [41]:
# 유전체 SNP 매칭
gen_dict = {3335: "SNP_A-2181021",
            214246: "SNP_A-1809518",
            262783: "SNP_A-2130710",
            35799: "SNP_A-2263153",
            154529: "SNP_A-2310995",
            118171: "SNP_A-1922415",
            182907: "SNP_A-2266073",
            122732: "SNP_A-1984271",
            283204: "SNP_A-2218697",
            217552: "SNP_A-1850320",
            335062: "SNP_A-4262878",
            335032: "SNP_A-4299800",
            335031: "SNP_A-2242511"}

In [47]:
# 유전체 데이터프레임 만들기
df_all_gen = pd.DataFrame()

for v in gen_dict.values():
    temp = pd.read_csv(f"./dataset/genome/result_{v}.csv", index_col=1, low_memory=False).dropna()
    temp.rename(columns={"SNP": v}, inplace=True)
    df_all_gen = pd.concat([df_all_gen, temp[v]], axis=1)

df_all_gen.head()

,SNP_A-2181021,SNP_A-1809518,SNP_A-2130710,SNP_A-2263153,SNP_A-2310995,SNP_A-1922415,SNP_A-2266073,SNP_A-1984271,SNP_A-2218697,SNP_A-1850320,SNP_A-4262878,SNP_A-4299800,SNP_A-2242511
NIH2307537078,C C,G A,G G,A A,C C,T T,G G,C C,A A,A G,T G,G G,A A
NIH2307151907,A A,G A,G T,A A,T C,T T,G G,G G,A A,G G,T G,G G,A A
NIH2307876488,C C,G G,T T,A A,C C,T T,G G,C C,G A,G G,T G,G G,A A
NIH2307009808,C C,G A,G T,A A,C C,T T,G G,G C,A A,G G,T G,G G,A A
NIH2307798369,C C,A A,T T,A A,C C,T T,G G,G C,G A,G G,G G,G G,A A


In [48]:
len(df_all_gen)

4658

결측값 찾기
```
SNP_A-2181021    26
SNP_A-1809518     2
SNP_A-2130710    15
SNP_A-2263153     0
SNP_A-2310995     8
SNP_A-1922415    36
SNP_A-2266073     0
SNP_A-1984271     0
SNP_A-2218697     0
SNP_A-1850320    40
SNP_A-4262878    12
SNP_A-4299800     1
SNP_A-2242511     0
dtype: int64
```

In [49]:
# df_all_gen= df_all_gen.replace('0 0', pd.NA)

mode_values = df_all_gen.mode().iloc[0]

df_all_gen= df_all_gen.replace('0 0', mode_values) # 최빈값으로 대치
df_all_gen.isnull().sum()

SNP_A-2181021    0
SNP_A-1809518    0
SNP_A-2130710    0
SNP_A-2263153    0
SNP_A-2310995    0
SNP_A-1922415    0
SNP_A-2266073    0
SNP_A-1984271    0
SNP_A-2218697    0
SNP_A-1850320    0
SNP_A-4262878    0
SNP_A-4299800    0
SNP_A-2242511    0
dtype: int64

In [50]:
diet_and_gen = pd.concat([binary, cath0, cath1, cnt, df_all_gen], axis=1)
diet_and_gen.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5233 entries, NIH2307713334 to NIH2307981982
Columns: 127 entries, AS1_SEX to SNP_A-2242511
dtypes: float64(114), object(13)
memory usage: 5.1+ MB


In [53]:
diet_and_gen.isnull().sum()

AS1_SEX          463
AS1_HT           463
AS1_DM           463
AS1_AL           463
AS1_MI           463
                ... 
SNP_A-2218697    575
SNP_A-1850320    575
SNP_A-4262878    575
SNP_A-4299800    575
SNP_A-2242511    575
Length: 127, dtype: int64

In [54]:
diet_and_gen.dropna(subset=['AS1_PMAG_C', 'AS1_PMYN_C']).info()
diet_and_gen.dropna(subset=df_all_gen.columns).info()
# diet_and_gen.dropna(subset=['AS1_PMYN_C'],).info()

<class 'pandas.core.frame.DataFrame'>
Index: 4770 entries, NIH2307713334 to NIH2307534318
Columns: 127 entries, AS1_SEX to SNP_A-2242511
dtypes: float64(114), object(13)
memory usage: 4.7+ MB
<class 'pandas.core.frame.DataFrame'>
Index: 4658 entries, NIH2307713334 to NIH2307981982
Columns: 127 entries, AS1_SEX to SNP_A-2242511
dtypes: float64(114), object(13)
memory usage: 4.5+ MB


In [55]:
diet_and_gen = diet_and_gen.dropna(subset=['AS1_PMAG_C', 'AS1_PMYN_C'])
print(len(diet_and_gen))
diet_and_gen = diet_and_gen.dropna(subset=df_all_gen.columns)
print(len(diet_and_gen))
diet_and_gen.info()

4770
4199
<class 'pandas.core.frame.DataFrame'>
Index: 4199 entries, NIH2307713334 to NIH2307534318
Columns: 127 entries, AS1_SEX to SNP_A-2242511
dtypes: float64(114), object(13)
memory usage: 4.1+ MB


In [56]:
dataset.head()

,AS1_SEX,AS1_AGE,AS1_FAMNUM,AS1_HAND,AS1_SEAS,AS1_MARRYA,AS1_MARRYAETC,AS1_RELIG,AS1_RELIGETC,AS1_RELIGPRT,...,AS1_DRAS,AS1_DRLP,AS1_DRSP,AS1_DRSL,AS1_FMOSREL_N,AS1_FMOSREL_P,AS1_FMOSREL_S,AS1_FMOSREL_O,AS1_BPSIT1SYS,AS1_BPSIT1DIA
DIST_ID,,,,,,,,,,,,,,,,,,,,,
NIH2307713334,2,44,2.0,1.0,2.0,2.0,NaN,4.0,NaN,NaN,...,0,0,0,0,0,0,0,0,100.0,70.0
NIH2307455306,2,43,6.0,1.0,2.0,2.0,NaN,3.0,NaN,1.0,...,0,0,0,0,0,0,0,0,120.0,80.0
NIH2307137554,2,64,4.0,1.0,1.0,2.0,NaN,3.0,NaN,1.0,...,0,0,0,0,0,0,0,0,174.0,102.0
NIH2307649667,2,65,2.0,1.0,3.0,2.0,NaN,2.0,NaN,2.0,...,0,0,0,0,0,0,0,0,126.0,82.0
NIH2307899915,2,57,6.0,1.0,3.0,2.0,NaN,3.0,NaN,2.0,...,0,0,0,0,0,1,0,0,102.0,74.0


In [57]:
len(dataset)

4798

In [58]:
X = diet_and_gen

# X의 인덱스와 dataset의 인덱스를 기준으로 겹치는 인덱스만 추출
common_index = X.index.intersection(dataset.index)
y = dataset['OP'].loc[common_index]

In [59]:
print("dataset[OP]:", len(dataset['OP']))
print("x", len(X), "y:", len(y))

dataset[OP]: 4798
x 4199 y: 4199


In [60]:
X.index

Index(['NIH2307713334', 'NIH2307455306', 'NIH2307137554', 'NIH2307649667',
       'NIH2307899915', 'NIH2307202612', 'NIH2307895121', 'NIH2307990429',
       'NIH2307188829', 'NIH2307769863',
       ...
       'NIH2307705241', 'NIH2307002485', 'NIH2307422716', 'NIH2307959786',
       'NIH2307294252', 'NIH2307635277', 'NIH2307591031', 'NIH2307808901',
       'NIH2307930134', 'NIH2307534318'],
      dtype='object', length=4199)

In [61]:
y.value_counts()

OP
0    3393
1     806
Name: count, dtype: int64

In [62]:
X.to_csv("dataset/X_24.ver.0.5.csv")
y.to_csv("dataset/y_24.ver.0.5.csv")

X, y만 파일로 내보내고, 폐경 여부는 다른 곳에서 처리

In [63]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [64]:
processsed_cnt = scaler.fit_transform(cnt)

In [65]:
cnt

cnt,AS1_AGE,AS1_TOTALC,AS1_PMAG_C,AS1_MNSAG,AS1_SLPAMTM,AS1_ENERGY,AS1_CARBO,AS1_FAT,AS1_PROTEIN,AS1_FIBER,...,AS1_WEIGHT,AS1_BMI,AS1_BDFTR,AS1_BDCMSC,AS1_BPSIT1LS,AS1_BPSIT1RS,AS1_BPSIT1SYS,AS1_BPSIT1LD,AS1_BPSIT1RD,AS1_BPSIT1DIA
DIST_ID,,,,,,,,,,,,,,,,,,,,,
NIH2307713334,44,0.00,0.0,13.0,8.0,1694.174446,325.969272,18.612650,52.250152,19.829582,...,58.0,24.809215,35.0,35.5,100.0,100.0,100.0,70.0,60.0,70.0
NIH2307455306,43,0.29,0.0,20.0,7.0,1518.742501,303.288539,11.561803,43.115031,19.166532,...,67.0,26.369274,33.8,41.9,120.0,118.0,120.0,80.0,78.0,80.0
NIH2307137554,64,0.00,38.0,17.0,7.0,3033.465161,519.516083,56.330170,117.972867,45.703598,...,48.0,21.333333,30.6,31.5,168.0,174.0,174.0,100.0,102.0,102.0
NIH2307649667,65,0.62,52.0,18.0,7.0,990.403175,199.319908,8.016458,25.462945,15.051000,...,66.0,28.641806,35.5,40.1,120.0,126.0,126.0,80.0,82.0,82.0
NIH2307899915,57,0.00,46.0,16.0,7.0,2087.710623,323.282165,52.200932,67.995519,20.968875,...,56.5,23.068522,27.1,38.8,102.0,100.0,102.0,70.0,74.0,74.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NIH2307591031,60,0.00,46.0,17.0,5.0,1363.519475,268.321021,12.078711,36.674718,12.458386,...,59.0,25.040053,26.7,40.2,120.0,140.0,140.0,80.0,80.0,80.0
NIH2307808901,62,0.00,42.0,18.0,5.0,1121.040518,167.326272,26.935002,49.041384,13.648075,...,60.0,26.666667,39.5,34.4,120.0,128.0,128.0,84.0,78.0,84.0
NIH2307930134,65,0.00,50.0,19.0,5.0,1568.944403,285.454118,21.977073,49.200527,15.514005,...,66.5,27.255841,33.6,41.2,118.0,120.0,120.0,80.0,90.0,90.0


In [66]:
processsed_cnt = pd.DataFrame(processsed_cnt, columns=cnt.columns, index=cnt.index)

In [67]:
# plt.tight_layout()
# # 75개의 바이올린 플롯을 위한 빈 Figure와 Axes 객체 생성
# fig, axes = plt.subplots(nrows=5, ncols=15, figsize=(50, 30))

# # 칼럼 인덱스 카운터
# i = 0

# # 모든 칼럼 순회
# for col in cnt.columns:
#     # 5행 15열 Axes 객체 배열에 순서대로 바이올린 플롯 그리기
#     sns.violinplot(data=processsed_cnt, x=col, ax=axes[i // 15][i % 15])
#     i += 1

# # 그래프 제목 설정
# fig.suptitle("[Continuous variables]")

# # x축 라벨 설정
# for ax in axes.flat:
#     ax.set_xlabel(ax.get_xlabel(), rotation=0)

# # y축 라벨 설정
# axes[0, 0].set_ylabel("[none]")

# # plt.show()를 사용하면 그래프가 즉시 뜨고 코드 실행이 멈춥니다.
# # plt.savefig("[파일 경로]")를 사용하면 지정된 경로에 그래프 파일을 저장합니다.
# plt.savefig("./Docs/scaled_vp.png")

In [68]:
print(dropped_col_binary)
print(dropped_col_cath0)
print(dropped_col_cath1)
print(dropped_col_cnt)

['AS1_HYST', 'AS1_OBGYOP', 'AS1_BRSTF']
['AS1_HYSTOVARYW', 'AS1_OVARYW']
[]
['AS1_CHILD_P', 'AS1_FPREGAG', 'AS1_VITB12', 'AS1_PRT_U', 'AS1_CREATINE_U', 'AS1_CA_U', 'AS1_NA_U', 'AS1_TSH']
